## Introduction

(TODO)

### "Task-level Value Affects Trial-level Reward Processing" - the original paper in brief

#### Experiment: 
- Participants were presented with **slot machines** ("trials") where they had to select one of two arms to pull. If the participant selected the correct arm on the slot machine, they would **"win"** at the machine and gain \\$0.03, while if they selected the wrong arm, they would **"lose"** the trial i.e. gain \\$0.00.
- The **win/loss feedback** for each slot machine was shown to them after they selected an arm. Their goal was to **win as much money as possible**.
- Each slot machine was preceded by a **cue** (coloured shape) shown to the participant. The slot machines were divided into three sets or **casinos** ("tasks").
- The different cues corresponded to **different types of slot machines**. Some machines, called **"low-value" machines**, had a 50% win probablility on both arms, while others called **"high-value machines"** had an 80% win probability on one arm and 20% on the other.
- For high-value machines, if the participant selected the 80%-arm it was considered a **"correct"** response. There was no "correct" response for a low-value machine.
- The **participants were not informed** of these differences. The **win amount was kept constant** across all slot machines.
- The **three casinos had a mix of high- and low- value machines** - one each of "high-value" and "low-value" task/casino containing only high-/low- value slot machines respectively, and a "mid-value" task containing equal numbers of both.
- Each of the 6 machine types was presented 24 times in random order (144 trials per task).

#### Research question: 
How does task-level reward processing affect trial-level value processing in ACC?

#### Data Preprocessing:
- Excluding participants who did not learn the cues - in order to be included, a participant should have **correct** responses for **at least 60% of the trials** in the mid and high value tasks.
- For each task, the first ten trials for each cue type were excluded, since it was assumed that the participants would still be learning the cues.

#### Analysis:
- The researchers looked into "Reward Positivity" (RewP), which corresponds to how much outcomes differ from learned action values \[1].
- They created the feedback-locked ERP (mean across epochs) for different task-cue-participant combinations. Then, they constructed difference waves between the mean win ERP and mean loss ERP across all participants for the task-cue combination.
- They examined the data from the FCz electrode between 240–340 ms post feedback, based on the recommendations from previous research on reward prediction. The RewP was defined as the maximum voltage within this window for each task-cue-participant.
- Analysis was carried out in MATLAB 2022a using the EEGLab library.


### Some notes on using the dataset
- Although the paper says the data was sampled at 250 Hz, according to the dataset the sampling rate is 1000 Hz and we have proceeded assuming this 1kHz value.
- The paper states that the 10-20 electrode montage was used.
- Due to participants' wishes, only part of the data was shared by the researchers - that collected at the Oxford testing site. This means that we have fewer subjects to analyse, and also that the notch filtering required will be only at one frequency (50 Hz at Oxford).
- Each subject's 'eeg' subfolder contains a 'sub-XX_task-casinos_events.json' containing names and descriptions of the events like onset of fixation cross, cue onset, response, and feedback. This list is not complete, namely some events like S12, S22, S32 for the cue onsets are missing. However, the events are present in the data and can be used directly for data processing.
- Event name format: Each event name consists of four characters, with spaces used for padding (e.g. ```S  6, S 16```). However in some forms of output (e.g. in jupyter notebooks that we used while initially exploring the data) it can show up as three characters (```S 6, S 16```) and in the json file mentioned above the names are also slightly different (```S6, S16```). 

### Authors' Pipeline vs. Our Pipeline

(TODO)

### Visual Inspection of raw data to find bad channels/subjects

(TODO)

### Removing Participants and Trials from the Analysis.

Before passing the data to the automated pipeline, we exclude some trials and participants according to the rules below: 
1. We are removing the first **n** trials for every cue in every casino. This ensures that the participants are able to learn the pattern of every slot as we want to measure how the RewP changes in different average task values. For that the participants have to learn first how the cues of every slot machine correspond to the reward. Here, we decided to reduce the number of trials excluded per machine type (cue) to 4 trials, since we have less data to work with overall.
2. We are removing participants that do not manage to get **threshold** of the learnable trials correct, i.e. choosing the lever with the higher win probability. Thus, we assume that participants did not truly care about the reward or did not manage to learn the cues. 

In [8]:
import os
import pandas as pd

# HYPERPARAMS
n = 4
threshold = 0.7

subjects_dir = 'Dataset/ds004147-filtered'  # copy dataset an add "-filtered" suffix
subject_dirs = [d for d in os.listdir(subjects_dir) if os.path.isdir(os.path.join(subjects_dir, d))]
removed_trial_dict = {}
for subject_dir in subject_dirs:
    beh_file_path = os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_task-casinos_beh.tsv')

    if os.path.exists(beh_file_path):
        df = pd.read_csv(beh_file_path, sep='\t')
        # remove the first four trials for every cue and block
        filtered_df = df.groupby(['block', 'cue']).apply(lambda x: x.iloc[4:]).reset_index(drop=True)
        filtered_df = filtered_df[filtered_df['invalid'] != 1]
        removed_trials = df.groupby(['block', 'cue']).apply(lambda x: x.iloc[:4]).reset_index(drop=True)
        removed_trials["trial"] = removed_trials["trial"] + (removed_trials["block"] - 1) * 144
        removed_trial_dict[subject_dir] = removed_trials
        df_sorted = filtered_df.sort_values(by=['block', 'trial'])
        # aggregate learnable trials and mark participants that did not meet the threshold
        learnable_trials = filtered_df[filtered_df['prob'] == 80]
        correct_choices = learnable_trials['outcome'] == learnable_trials['optimal']
        success_rate = correct_choices.mean()
        threshold_met = success_rate >= threshold
        if threshold_met:
            print(f"{subject_dir} does meet the threshold and should be included in the analysis.")
            df_sorted.to_csv(os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_task-casinos_beh.tsv'), sep='\t', index=False)
        else:
            print(f"{subject_dir} does  not meet the threshold and should be not included in the analysis.")
            df_sorted.to_csv(os.path.join(subjects_dir, subject_dir, 'beh', f'{subject_dir}_REJECT_task-casinos_beh.tsv'), sep='\t', index=False)
    else:
        print(f'Behavioral file for {subject_dir} does not exist.')

Behavioral file for code does not exist.
sub-27 does meet the threshold and should be included in the analysis.
sub-28 does meet the threshold and should be included in the analysis.
sub-29 does meet the threshold and should be included in the analysis.
sub-30 does meet the threshold and should be included in the analysis.
sub-31 does meet the threshold and should be included in the analysis.
sub-32 does meet the threshold and should be included in the analysis.
sub-33 does meet the threshold and should be included in the analysis.
sub-34 does meet the threshold and should be included in the analysis.
sub-35 does meet the threshold and should be included in the analysis.
sub-36 does meet the threshold and should be included in the analysis.
sub-37 does meet the threshold and should be included in the analysis.
sub-38 does meet the threshold and should be included in the analysis.



After choosing n=4 and threshold=0.7, no participants went under the threshold. The authors noted that some were excluded, so we believe that these participants where from the other test site.

### Filter the first n=4 trials for every slot in every casino for every participant

During the removal of bad participants, we also returned a filtered beh.tsv without the first n=4 trials. We do this for the eeg.vmrk and events.tsv files, respectively. Moreover, invalid trials are also removed, as the data is unnecessary.


In [9]:
from mne_bids import (BIDSPath,read_raw_bids)
import mne
import tqdm

# path where to save the datasets.
bids_root = subjects_dir

for subject_dir in subject_dirs:
    eeg_vmrk_path = os.path.join(subjects_dir, subject_dir, 'eeg', f'{subject_dir}_task-casinos_eeg.vmrk')
    eee_events_tsv_path = os.path.join(subjects_dir, subject_dir, 'eeg', f'{subject_dir}_task-casinos_events.tsv')
    if os.path.exists(eeg_vmrk_path):
        with open(eeg_vmrk_path, 'r') as file:
            lines = file.readlines()
        start_index = 13  
        groups = []
        current_group = []
        for i, line in enumerate(lines[start_index:], start=start_index):
            if any(stim in line for stim in ['S  1', 'S 11', 'S 21', 'S 31']):
                if current_group:
                    groups.append(current_group)
                    current_group = []
            current_group.append(i)
        if current_group:
            groups.append(current_group)
        groups = [(idx+1, element) for idx, element in enumerate(groups)]
        invalid = [(idx, element) for idx, element in groups if len(element) != 5]  # filter invalid trials, i.e. sublist != 5
        print(f"Subject: {subject_dir} had {len(invalid)} invalid trials!")
        # filter the 4 trials for every slot in every casino
        first_four = [(idx, item) for idx, item in groups if idx in removed_trial_dict[subject_dir]["trial"].values]
        first_four_invalid= [item for item in first_four if len(item[1]) != 5]
        # remove duplicate in invalid (invalid trials could be in the first four trials of a slot)
        invalid = [item for item in invalid if item not in first_four_invalid]
        first_four.extend(invalid)
        # convert to list of lines to remove
        first_four = [element[1] for element in first_four]
        lines_to_remove = []
        [lines_to_remove.extend(element) for element in first_four]
        filtered_lines = [line for idx, line in enumerate(lines) if idx not in lines_to_remove]
        
        # create filtered vrmk file
        #print(f"Adapting eeg.vmrk and events.tsv file for {subject_dir}")
        with open(os.path.join(subjects_dir, subject_dir, 'eeg', f'{subject_dir}_task-casinos_eeg.vmrk'), "w") as file:
            # Write each item to the file
            for item in filtered_lines:
                file.write(item)
        
        
        # do the same for events.tsv file
        with open(eee_events_tsv_path, 'r') as file:
            lines = file.readlines()
        lines_to_remove = [idx - 10 for idx in lines_to_remove]
        filtered_lines = [line for idx, line in enumerate(lines) if idx not in lines_to_remove]
        with open(os.path.join(subjects_dir, subject_dir, 'eeg', f'{subject_dir}_task-casinos_events.tsv'), "w") as file:
            # Write each item to the file
            for item in filtered_lines:
                file.write(item)
    else:
        print(f'eeg.vmrk file for {subject_dir} does not exist.')

eeg.vmrk file for code does not exist.
Subject: sub-27 had 5 invalid trials!
Subject: sub-28 had 2 invalid trials!
Subject: sub-29 had 91 invalid trials!
Subject: sub-30 had 19 invalid trials!
Subject: sub-31 had 7 invalid trials!
Subject: sub-32 had 59 invalid trials!
Subject: sub-33 had 25 invalid trials!
Subject: sub-34 had 16 invalid trials!
Subject: sub-35 had 12 invalid trials!
Subject: sub-36 had 29 invalid trials!
Subject: sub-37 had 16 invalid trials!
Subject: sub-38 had 7 invalid trials!


#### Note: 

We can draw a graph for the number of invalid trials by subject. Subject 29 had nearly 25% of the trials incorrect, however we have included the subject in the analysis as of now.

### Pipeline Configuration - thought process

#### Downsampling:

To save on processing power required, we had initially downsampled the data from 1000 Hz to 150 Hz. ```mne-bids-pipeline``` constructs the PSD plot for frequencies up to 1.5 * lowpass_cutoff_frequency. Thus 150 Hz is sufficient to allow us to include all required frequencies in our analysis.
However, the performance after resampling is also not noticeably better than without resampling, so we opted not to resample to avoid losing out on raw data.

#### Filtering:

- We have used a band-pass filter (0.1 - 50Hz) since frequencies of interest for EEG analysis would lie within this band.
- A notch filter at 50Hz is applied (power line frequency at the Oxford site).
- If the data from the other site had also been available, we could have run the filtering step twice with the respective notch frequencies on the two groups of subjects by specifying the subject IDs in the configuration file. Then we could proceed with the next steps running on all subjects, since mne-bids-pipeline caches the intermediate results of processing steps.

#### Re-Referencing:

The authors re-referenced to the average of the mastoid signals, using the "TP9" and "TP10" channels. 
Since this is a common rereferencing scheme, we have used the same, by setting the relevant parameter in the config file as follows:
```eeg_reference = ['TP9', 'TP10']``` .

#### Epochs

Following the paper, we have selected epochs as follows:
- ICA training - cue-locked, 0 to 3 seconds 
- ERP analysis - feedback-locked, -200ms to +600ms

#### Events

For this analysis, we selected the events related to win/loss feedback, i.e. S6, S16, S26, S36, S7, S17, S27, S37. These can be specified in the 'conditions' parameter of the configuration file. We can also rename the events for human-readability, specifying the event type - cue, beep, ..., win (feedback), loss (feedback) along with the task-cue combination - Low-Low (LL), Mid-Low (ML) and so on. There is a custom parameter ```renaming_flag``` which if set as ```True```, will rename the events.

In future, the pipeline can also be run for a different set of events by changing the 'conditions' parameter, e.g. we can analyse cue-locked waveforms rather than feedback-locked as we have done here, by including cue events even during the post-ICA step.

#### ICA for artefact removal:

The authors of the paper used different epochs for ICA (0-3 seconds starting at cue event) and for ERP  analysis (-0.2 to +06. second epochs locked to feedback events).  In a previous version of the pipeline we were using (1.5), epoching was done before ICA, whereas in the latest version v1.7, the epoching step can be explicitly run after the ICA steps. 

For our pipeline, we have a variable ```ica_train_step``` (default False) which should be set to True when performing the ICA and back to False after the ICA training is done. This will cause the relevant settings like event names ("conditions") and epoch length to be set accordingly.

#### Peak-to-Peak based rejection

In the paper, the authors rejected an epoch from analysis if it had more than 150 microvolts of overall change across the epoch. We have done the same via the ```reject``` parameter.

#### Evoked potentials

(TODO) - conditions and contrasts


### Selecting ICA components

(TODO)

### Try it Yourself!

These are the steps to run the analysis from start to end:

1. **Download the dataset and extract it** to a folder of your choice. For us, the main project folder was "EEG2324Brownie" and the dataset in the folder 'Dataset/ds004147' inside it.
2. **Removing participants, invalid trials, and learning trials** from the dataset - run the script __ (TODO name) from ```utils.py```. This creates a new folder 'Dataset/ds004147-filtered/', which is the folder to use for the actual analysis.  
3. **Initialisation**: In the config file ```brownie-config.py```, set the values of parameters like study_name (main project folder - for us, 'EEG2324Brownie'), bids_root (should point to the new 'ds004147-filtered' folder), subjects (if you want to analyse only a few subjects vs. all), etc. (TODO check full list after cleaning up config file)
4. Filtering and ICA step:
    - Set ```ica_train_step = True``` in the configuration file
    - Run the initial pipeline steps from the command line:
      ```mne_bids_pipeline --config=./brownie-config.py --steps=init,preprocessing/_01_data_quality,preprocessing/_04_frequency_filter,preprocessing/_06a1_fit_ica,preprocessing/_06a2_find_ica_artifacts ```
    - The pipeline will create a new folder 'derivatives/mne_bids_pipeline' within the 'ds004147-filtered' folder (if it doesn't exist already) and store its outputs there. 
5. ICA component inspection: For each subject analysed, open the corresponding folder in the derivatives section and inspect the report at sub-XX_task-casinos_proc-ica+components_report.html. In sub-XX_task-casinos_proc-ica_components.tsv you can mark individual ICA components as 'bad' or 'good' depending on what the pipeline has auto-classified as well as from personal judgement, and then save this file to be used for the next step.
6. Continue with epoching and apply ICA:
   - set ```ica_train_step = False``` to make sure analysis epochs are correctly created
   - Run ```mne_bids_pipeline --config=./brownie-config.py --steps=preprocessing/_07_make_epochs,preprocessing/_08a_apply_ica```
   - You can see the output of the ICA cleaning in sub-XX_task-casinos_report.html
8. Peak-to-Peak rejection: run ```mne_bids_pipeline --config=./brownie-config.py --steps=preprocessing/_09_ptp_reject```
9. Evoked Potentials:
   - run ```mne_bids_pipeline --config=./brownie-config.py --steps=sensor/_01_make_evoked```
   - run (TODO function name/ create cell to run this) to load and plot the evoked data. If you want to view only a subset of channels (e.g. only FCz), change the list in 'picks' before plotting.

### Further Analysis

(TODO)

#### Setup

We used mne-bids-pipeline v1.7.0, running on Windows and on Ubuntu Linux. The environment can be replicated using the environment file available in the git of this project. 

#### References

Paper: https://doi.org/10.1016/j.neuroimage.2022.119456.

BIDS data format: doi:10.1038/sdata.2016.44

